In [31]:
#pip install tensorflow


In [36]:
#pip install deepface


In [37]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [26]:
import os
import cv2
import numpy as np
import tensorflow as tf
#from retinaface import RetinaFace

from tensorflow.keras.models import load_model
from deepface import DeepFace

from mtcnn import MTCNN
import math
import mediapipe as mp
from tensorflow.keras.preprocessing.image import load_img, img_to_array

24-04-09 11:40:36 - Directory C:\Users\gotti/.deepface created
24-04-09 11:40:36 - Directory C:\Users\gotti/.deepface/weights created


ModuleNotFoundError: No module named 'mediapipe'

In [2]:
lwcnn = load_model("C:/Users/Charanteja/Design_project/lwcnn_v4.h5")


In [3]:
def channel_wise_histogram_equalization(image):
    # Split the image into its color channels
    b, g, r = cv2.split(image)

    # Apply histogram equalization to each color channel
    equalized_b = cv2.equalizeHist(b)
    equalized_g = cv2.equalizeHist(g)
    equalized_r = cv2.equalizeHist(r)

    # Merge the equalized color channels
    equalized_image = cv2.merge((equalized_b, equalized_g, equalized_r))

    return equalized_image

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from keras.preprocessing.image import img_to_array

# Function to detect faces and check for consistent "drowsy" prediction
def detect_faces_with_alarm():
    # Initialize MediaPipe Face Detection
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection()

    classes = ['active','drowsy','yawning']

    # Flag to track consecutive 'drowsy' predictions
    drowsy_count = 0

    # Flag to indicate whether alarm is sounding
    alarm_on = False

    cam2 = cv2.VideoCapture(0)
    while True:
        ret, image = cam2.read()
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Perform face detection on the image
        results = face_detection.process(image_rgb)

        # Draw bounding boxes around detected faces
        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = image.shape

                x, y, w, h = (
                    int(bboxC.xmin * iw),
                    int(bboxC.ymin * ih),
                    int(bboxC.width * iw),
                    int(bboxC.height * ih),
                )
                x, y, w, h = max(0, x), max(0, y), min(iw - x, w), min(ih - y, h)

                # Change color if label is 'drowsy'
                color = (0, 255, 0)  # Default color
                text_color = (0, 255, 0)  # Default text color

                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                face = image[y:y + h, x:x + w]
                face = channel_wise_histogram_equalization(face)
                face = cv2.resize(face, (224, 224))
                face = face.reshape(224, 224, 3)
                face = np.array(img_to_array(face))
                face = face / 255.0
                face = np.expand_dims(face, axis=0)

                # Make predictions
                pred = lwcnn.predict(face)
                predicted_score = round(np.max(pred) * 100, 2)
                score_text = f'{predicted_score}%'

                predicted_class = np.argmax(pred, axis=1)
                label = classes[int(predicted_class)]
                
                # Change color to red if label is 'drowsy'
                if label == 'drowsy':
                    color = (0, 0, 255)
                    text_color = (0, 0, 255)
                    drowsy_count += 1
                    if drowsy_count >= 5:  # If 'drowsy' for 5 consecutive frames or more
                        alarm_on = True
                        # Your code to activate alarm (e.g., sound an alarm)
                        print("Drowsiness detected for 5 frames or more! Sound the alarm!")

                else:
                    drowsy_count = 0
                    alarm_on = False
                    # Your code to deactivate alarm (e.g., stop the alarm sound)
                
                # Display text with the appropriate color
                # cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, text_color, 2)
                cv2.putText(image, f'{label} ({score_text})', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, text_color, 2)

        cv2.imshow('Real_Time_Driver_Drowsiness_Detection', image)
        
        # If alarm is on, sound it for 5 seconds
        if alarm_on:
           
            pass

        if cv2.waitKey(27) & 0xFF == ord('q'):
            break

    cam2.release()
    cv2.destroyAllWindows()

detect_faces_with_alarm()
